In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

In [2]:
my_path = '/Users/munjikahalah/code/market_sentiment/raw_data/dump.json'

In [3]:
df = pd.read_json(my_path, nrows=1000, lines=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   essid                      1000 non-null   object             
 1   wit                        1000 non-null   object             
 2   custom_query               1000 non-null   object             
 3   id_str                     1000 non-null   int64              
 4   created_at                 1000 non-null   datetime64[ns, UTC]
 5   full_text                  1000 non-null   object             
 6   source                     1000 non-null   object             
 7   user_id_str                1000 non-null   int64              
 8   lang                       1000 non-null   object             
 9   favorite_count             1000 non-null   int64              
 10  reply_count                1000 non-null   int64              
 11  retwe

In [5]:
df.columns

Index(['essid', 'wit', 'custom_query', 'id_str', 'created_at', 'full_text',
       'source', 'user_id_str', 'lang', 'favorite_count', 'reply_count',
       'retweet_count', 'quote_count', 'limited_actions', 'geo', 'place',
       'coordinates', 'conversation_id_str', 'in_reply_to_status_id_str',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name',
       'quoted_status_id_str', 'forward_pivot', 'symbols', 'name',
       'screen_name', 'datetimestamp', 'urls', 'quoted_user_id_str',
       'quoted_user_screen_name', 'media', 'hashtags', 'user_mentions'],
      dtype='object')

In [6]:
cleaned_df = df[["full_text","favorite_count","symbols","datetimestamp","created_at","lang"]]

In [7]:
pd.set_option("display.max_colwidth", None) 

In [8]:
# cleaning means removing \n 
cleaned_df[['full_text']] 

,full_text
0,Расцвет человеческого общества зависит от двух факторов: наличия интеллектуальной мощи выдающихся людей и способности этих и других людей сделать их идеологии приятными для большинства.\n\n$btc\n\n $alts $wan $strat $mtl $nano $nkn $kmd $zrx $zec $zen $loki $bat $gnt $dnt
1,I’m waiting for the volatility to settle with $btc
2,Buy $BTC this is not a fucking drill
3,"$btc $btcusd nvm, 8,8k target hit"
4,"Mar 8, 2020 03:12:00 UTC The price of $BTC currently is $8762.10"
...,...
995,14時51分 \n仮想通貨取引所Binance出資の SafePal ハードウォレットが発売開始!\n現在SFPは次のエアドロはOOE!ハードウォレット保有者エアドロが申請できる!\n\n$BTC $FLOW $SFP \n Binance通貨ペア:SFP/USDT\n 現在価格:$3.338\nhttps://t.co/1kIoubGHBV
996,He posts paid recommendations for free! He has 44X for month\nI am stunned at the profit. You should follow right now @gem_detecter\n\n $mana $nexo $zil $klay $ele $btcb $eth $vet $xem $stx $rbc $free $btg $sol $celo $dat $trx $btmx $dent $leo $xmr $buy $eos $uni $btc $dgb $cake
997,Unusual #volume spike in $FTTBTC\n\n17 times the average volume in $FTT - $BTC pair
998,ang sarap niyoooo $ETH $BTC


# Let's clean up the full text

In [9]:
def replacen(x):
    return x.replace("\n","")
def replacer(x):
    return x.replace("\r","")
def replacern(x):
    return x.replace("\r\n","")
    

cleaned_df["full_text"] = cleaned_df["full_text"].apply(replacen)
cleaned_df["full_text"] = cleaned_df["full_text"].apply(replacer)
cleaned_df["full_text"] = cleaned_df["full_text"].apply(replacern)

/var/folders/fw/j6mc6mcd20379_z864ff2bfw0000gn/T/ipykernel_7110/3674915676.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["full_text"] = cleaned_df["full_text"].apply(replacen)
/var/folders/fw/j6mc6mcd20379_z864ff2bfw0000gn/T/ipykernel_7110/3674915676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["full_text"] = cleaned_df["full_text"].apply(replacer)
/var/folders/fw/j6mc6mcd20379_z864ff2bfw0000gn/T/ipykernel_7110/3674915676.py:11: SettingWithCopyWarning: 
A value is trying 

In [10]:
import re
def remove_urls(text):
    text = re.sub(r'https?://\S+', '', text)
    return text

In [11]:
def remove_mention(text):
    text = re.sub(r'@\S+',"", text)
    return text

In [12]:
cleaned_df["full_text"] = cleaned_df["full_text"].apply(remove_urls)

/var/folders/fw/j6mc6mcd20379_z864ff2bfw0000gn/T/ipykernel_7110/3155825872.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["full_text"] = cleaned_df["full_text"].apply(remove_urls)


In [13]:
cleaned_df['full_text'] = cleaned_df['full_text'].apply(remove_mention)

/var/folders/fw/j6mc6mcd20379_z864ff2bfw0000gn/T/ipykernel_7110/2187274162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['full_text'] = cleaned_df['full_text'].apply(remove_mention)


# Embedding the full text

<p>The <b>Universal Sentence Encoder</b> encodes text into high dimensional vectors that can be used for text classification, semantic similarity, clustering, and other natural language tasks.  </p>

In [14]:
# Compute similarity matrix. Higher score indicates greater similarity.

In [15]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

2021-08-31 17:52:38.512164: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-31 17:52:40.842299: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [16]:
text_result = embed(cleaned_df['full_text'])

In [17]:
text_result

<tf.Tensor: shape=(1000, 512), dtype=float32, numpy=
array([[-3.8991722e-05, -4.8648175e-02, -4.3248527e-02, ...,
        -9.4580296e-03,  1.3665353e-02,  5.4488266e-03],
       [-8.8084355e-04,  6.8153441e-03, -1.0401447e-01, ...,
        -1.2548758e-02, -1.2986713e-02,  7.2317384e-02],
       [ 5.6798907e-04,  2.9366117e-02, -8.4870160e-02, ...,
        -4.3410663e-02, -1.6758954e-02,  6.1772242e-02],
       ...,
       [-1.2906455e-02,  4.4212304e-02, -7.6317273e-02, ...,
         4.6992496e-02,  3.0902388e-02, -2.3589913e-02],
       [ 6.2296912e-02,  5.5472460e-02, -5.7971947e-02, ...,
         1.9577026e-02, -1.9346960e-02,  8.5633971e-02],
       [-2.0291638e-02, -4.8588816e-02,  4.3813037e-03, ...,
        -4.1874420e-02,  4.1799083e-02, -1.6039994e-02]], dtype=float32)>

In [18]:
our_matrix = np.inner(text_result, text_result)

In [19]:
our_matrix

array([[0.9999998 , 0.1860455 , 0.24620427, ..., 0.2873267 , 0.278975  ,
        0.21145463],
       [0.1860455 , 1.        , 0.49706998, ..., 0.5015484 , 0.4572426 ,
        0.3117342 ],
       [0.24620427, 0.49706998, 0.9999999 , ..., 0.46174467, 0.46286118,
        0.30303806],
       ...,
       [0.2873267 , 0.5015484 , 0.46174467, ..., 0.9999996 , 0.31949338,
        0.23571685],
       [0.278975  , 0.4572426 , 0.46286118, ..., 0.31949338, 1.0000001 ,
        0.2848332 ],
       [0.21145463, 0.3117342 , 0.30303806, ..., 0.23571685, 0.2848332 ,
        0.99999964]], dtype=float32)

In [20]:
import yfinance as yf

In [21]:
tickers = yf.download(tickers = 'INTC BYND GE BTC', interval='1d',start="2018-01-01", end="2021-08-01")

[*********************100%***********************]  4 of 4 completed


In [27]:
difference = tickers['Close']['GE'].diff()

In [26]:
def one_or_zero(number):
    if number < 0:
        return 0
    else:
        return 1 

In [29]:
difference.apply(one_or_zero)

Date
2018-01-02    1
2018-01-03    1
2018-01-04    1
2018-01-05    1
2018-01-08    0
             ..
2021-07-26    1
2021-07-27    1
2021-07-28    1
2021-07-29    1
2021-07-30    0
Name: GE, Length: 901, dtype: int64

In [31]:
ge_stock = tickers['Close']['GE']

In [33]:
the_difference = ge_stock.diff()

In [38]:
the_difference.index

DatetimeIndex(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
               '2018-01-08', '2018-01-09', '2018-01-10', '2018-01-11',
               '2018-01-12', '2018-01-16',
               ...
               '2021-07-19', '2021-07-20', '2021-07-21', '2021-07-22',
               '2021-07-23', '2021-07-26', '2021-07-27', '2021-07-28',
               '2021-07-29', '2021-07-30'],
              dtype='datetime64[ns]', name='Date', length=901, freq=None)

In [37]:
cleaned_df.dtypes

full_text                      object
favorite_count                  int64
symbols                        object
datetimestamp                   int64
created_at        datetime64[ns, UTC]
lang                           object
dtype: object